# Leto Demo Notebook

# Setup

### Move to root of repo to run code. 

In [2]:
cd ..

/root/leto


In [3]:
!pip install aEye

### Setup env using script

In [ ]:
!./reconstruction_setup.sh

### Install all Requirements 

In [ ]:
!pip install -r requirements_aEye.txt

### Imports

In [5]:
import boto3
import cv2
import logging
import sys
from aEye.video import Video
from aEye.labeler import Labeler
from aEye.auxiliary import Aux
from aEye.RealBasicVSR_reconstruction_module.builder import download_model
root = logging.getLogger()
root.setLevel(logging.INFO)
root.setLevel(logging.DEBUG)
root.setLevel(logging.ERROR)

/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


### Create objects to process original video into the correct sizing

In [8]:
aux = Aux()

In [9]:
labeler = Labeler()

# Section 2: Downsampling trimmed video

### Grab the original Video from S3

In [10]:
video_list_s3 = aux.load_s3(bucket = 'leto-dish', prefix = 'original-videos/random-videos/')

In [11]:
print(video_list_s3)

[demo_10_second_clip.mp4, fun.mp4]


In [12]:
#print the metadata
print(video_list_s3[0].extract_metadata())

{'streams': [{'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'High', 'codec_type': 'video', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 1920, 'height': 1080, 'coded_width': 1920, 'coded_height': 1080, 'closed_captions': 0, 'film_grain': 0, 'has_b_frames': 2, 'pix_fmt': 'yuv420p', 'level': 40, 'color_range': 'tv', 'color_space': 'bt709', 'color_transfer': 'bt709', 'color_primaries': 'bt709', 'chroma_location': 'left', 'field_order': 'progressive', 'refs': 1, 'is_avc': 'true', 'nal_length_size': '4', 'id': '0x1', 'r_frame_rate': '25/1', 'avg_frame_rate': '25/1', 'time_base': '1/12800', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 128000, 'duration': '10.000000', 'bit_rate': '79042', 'bits_per_raw_sample': '8', 'nb_frames': '250', 'extradata_size': 50, 'disposition': {'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0, 'visual_impair

From the metadata we can see that the original codec is h.264, and the video is 1080p

### Downsample the videos

In [13]:
downsampled_video = labeler.change_resolution(video_list_s3,"240p")

In [14]:
aux.execute_label_and_write_local(downsampled_video, "reduced_videos")

[resized_426x240_demo_10_second_clip.mp4, resized_426x240_fun.mp4]

### Save to S3

In [24]:
aux.upload_s3(downsampled_video, bucket = 'leto-dish', prefix = 'reduced-videos/')

# Section 3: Reconstruction Video

Here we will use the realbasicvsr module to super resolve the video 4x.

### Download the pretrained model

In [15]:
download_model("aEye/RealBasicVSR_reconstruction_module/RealBasicVSR_x4.pth","leto-dish","pretrained-models/RealBasicVSR_x4.pth")

Downloaded file to: aEye/RealBasicVSR_reconstruction_module/RealBasicVSR_x4.pth


Please provide some time for the model to be downloaded. It is a 200mb pretrained model. :)

### Run the reconstruction_realbasicvsr.py giving inputs and outputs

In [9]:
from GPUtil import showUtilization as gpu_usage
gpu_usage() 

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [7]:
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()  

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  2% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% |  2% |


In [8]:
%%time

!python reconstruction_realbasicvsr.py /root/aEye/reduced_videos/resized_426x240_demo_10_second_clip.mp4 /root/aEye/reconstructed_videos/reconstructed_demo.mp4

/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
2023-07-14 21:47:41,942 - mmedit - INFO - load checkpoint from torchvision path: torchvision://vgg19
load checkpoint from local path: aEye/RealBasicVSR_reconstruction_module/RealBasicVSR_x4.pth
/opt/conda/lib/python3.7/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Traceback (most

In [24]:
pprint.pprint(torch.cuda.memory_summary(device=None, abbreviated=False))

('|===========================================================================|\n'
 '|                  PyTorch CUDA memory summary, device ID 0                 '
 '|\n'
 '|---------------------------------------------------------------------------|\n'
 '|            CUDA OOMs: 0            |        cudaMalloc retries: 0         '
 '|\n'
 '|===========================================================================|\n'
 '|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  '
 '|\n'
 '|---------------------------------------------------------------------------|\n'
 '| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  '
 '|\n'
 '|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  '
 '|\n'
 '|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  '
 '|\n'
 '|---------------------------------------------------------------------------|\n'
 '| Active memory         |       0 B  |       0 B  |       0 B

### Save reconstructed video to S3

In [10]:
recon_vid_obj = aux.load_local("/root/aEye/reconstructed_videos/upsampled_demo.mp4")

In [12]:
aux.set_local_path("/root/aEye/reconstructed_videos/")

In [13]:
aux.upload_s3(recon_vid_obj, bucket = 'leto-dish', prefix = 'reconstructed-videos/')

In [14]:
aux.clean()